# Edge Quality Analysis
Post-hoc analysis of visibility graph edge quality after a GTSFM pipeline run.

Loads `edge_quality_report.json` and `cluster_tree.pkl` from a completed run,
then visualizes bad edges, cluster structure, and the effect of pruning.

In [ ]:
from pathlib import Path

# === CONFIGURE THESE ===
OUTPUT_ROOT = Path("../results")  # --output_root used for the pipeline run
PRUNED_OUTPUT_ROOT = Path("../results_pruned")  # --output_root for pruned run
IMAGES_DIR = Path("../benchmarks/gerrard-hall/images")  # directory with source images

# Derived paths
EDGE_QUALITY_JSON = OUTPUT_ROOT / "results" / "edge_quality_report.json"
CLUSTER_TREE_PKL = OUTPUT_ROOT / "results" / "cluster_tree.pkl"
PRUNED_EDGE_QUALITY_JSON = PRUNED_OUTPUT_ROOT / "results" / "edge_quality_report.json"
PRUNED_CLUSTER_TREE_PKL = PRUNED_OUTPUT_ROOT / "results" / "cluster_tree.pkl"

In [ ]:
import json
import pickle

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# Load edge quality report
with open(EDGE_QUALITY_JSON) as f:
    report = json.load(f)

# Load cluster tree
with open(CLUSTER_TREE_PKL, "rb") as f:
    cluster_tree = pickle.load(f)

# Extract image filenames (added in our export update)
image_filenames = report.get("image_filenames", [])

# Parse edge quality into dict of (i,j) -> stats
edge_quality = {}
for edge_str, stats in report["edge_quality"].items():
    i, j = map(int, edge_str.strip("()").split(","))
    edge_quality[(i, j)] = stats

bad_edges = set()
for s in report["bad_edges"]:
    i, j = map(int, s.strip("()").split(","))
    bad_edges.add((i, j))

metadata = report["metadata"]
print(f"Total edges: {metadata['total_edges']}")
print(f"Bad edges:   {metadata['bad_edge_count']}")
print(f"Edges with no tracks: {metadata['edges_with_no_tracks']}")
print(f"Image filenames available: {len(image_filenames) > 0}")

## 1. Summary Statistics

In [ ]:
errors = []
track_counts = []
for stats in edge_quality.values():
    val = stats["mean_reproj_error_px"]
    if val != "inf":
        errors.append(val)
    track_counts.append(stats["num_tracks"])

errors = np.array(errors)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of mean reprojection errors
ax = axes[0]
ax.hist(errors, bins=50, edgecolor="black", alpha=0.7)
ax.axvline(5.0, color="red", linestyle="--", label="Bad threshold (5px)")
ax.set_xlabel("Mean Reprojection Error (px)")
ax.set_ylabel("Count")
ax.set_title("Distribution of Edge Reprojection Errors")
ax.legend()

# Histogram of track counts
ax = axes[1]
ax.hist(track_counts, bins=50, edgecolor="black", alpha=0.7, color="steelblue")
ax.set_xlabel("Number of Supporting Tracks")
ax.set_ylabel("Count")
ax.set_title("Distribution of Track Counts per Edge")

plt.tight_layout()
plt.show()

print(f"Mean reproj error:  {np.mean(errors):.2f} px")
print(f"Median reproj error: {np.median(errors):.2f} px")
print(f"Max reproj error:   {np.max(errors):.2f} px")
print(f"Edges > 5px: {np.sum(errors > 5.0)} / {len(errors)}")
print(f"Edges > 3px: {np.sum(errors > 3.0)} / {len(errors)}")
print(f"Edges > 1px: {np.sum(errors > 1.0)} / {len(errors)}")

## 2. Bad Edge Image Pairs
Showing the actual image pairs for the worst edges.

In [ ]:
def get_fname(idx):
    """Get image filename from index."""
    if idx < len(image_filenames):
        return image_filenames[idx]
    return f"image_{idx:04d}.jpg"


def load_img(img_dir, fname):
    """Load image, trying common extensions if exact match fails."""
    path = img_dir / fname
    if path.exists():
        return np.array(Image.open(path))
    for ext in [".jpg", ".jpeg", ".png", ".JPG", ".PNG"]:
        candidate = img_dir / (Path(fname).stem + ext)
        if candidate.exists():
            return np.array(Image.open(candidate))
    print(f"  WARNING: Could not find {fname} in {img_dir}")
    return np.zeros((100, 100, 3), dtype=np.uint8)


# Sort bad edges by error (worst first)
bad_edge_list = sorted(
    bad_edges,
    key=lambda e: edge_quality[e]["mean_reproj_error_px"]
    if edge_quality[e]["mean_reproj_error_px"] != "inf"
    else float("inf"),
    reverse=True,
)

n_show = min(len(bad_edge_list), 10)

if n_show > 0:
    fig, axes = plt.subplots(n_show, 2, figsize=(12, 4 * n_show))
    if n_show == 1:
        axes = axes[np.newaxis, :]

    for row, (i, j) in enumerate(bad_edge_list[:n_show]):
        stats = edge_quality[(i, j)]
        fname_i, fname_j = get_fname(i), get_fname(j)

        axes[row, 0].imshow(load_img(IMAGES_DIR, fname_i))
        axes[row, 0].set_title(f"[{i}] {fname_i}", fontsize=10)
        axes[row, 0].axis("off")

        axes[row, 1].imshow(load_img(IMAGES_DIR, fname_j))
        axes[row, 1].set_title(f"[{j}] {fname_j}", fontsize=10)
        axes[row, 1].axis("off")

        err = stats["mean_reproj_error_px"]
        err_str = f"{err:.1f}" if err != "inf" else "inf"
        axes[row, 0].set_ylabel(
            f"({i},{j}) err={err_str}px tracks={stats['num_tracks']}",
            fontsize=10, color="red", rotation=0, labelpad=120, va="center",
        )

    plt.tight_layout()
    plt.show()
else:
    print("No bad edges found!")

## 3. Graph Visualization
Interactive Plotly plots showing the visibility graph, clusters, and bad edges.

In [ ]:
# Graph visualization utilities (from partitioning.ipynb)
import plotly.graph_objects as go
from collections import deque
from typing import Any, Dict, List, Set, Tuple


def get_edge_coordinates(xy, edges):
    """Prepare edge coordinates for a Plotly Scatter trace."""
    if edges.size == 0:
        return np.array([]), np.array([])
    xe = np.empty(3 * len(edges))
    ye = np.empty(3 * len(edges))
    xe[0::3] = xy[edges[:, 0], 0]
    ye[0::3] = xy[edges[:, 0], 1]
    xe[1::3] = xy[edges[:, 1], 0]
    ye[1::3] = xy[edges[:, 1], 1]
    xe[2::3] = np.nan
    ye[2::3] = np.nan
    return xe, ye


def create_base_figure_with_background(xy, edges_arr):
    """Creates a Plotly figure with all nodes and edges as a faint background."""
    fig = go.Figure()
    xe_bg, ye_bg = get_edge_coordinates(xy, edges_arr)
    fig.add_trace(go.Scatter(
        x=xe_bg, y=ye_bg, mode="lines",
        line=dict(width=1, color="lightgray"),
        opacity=0.2, hoverinfo="none", showlegend=False,
    ))
    fig.add_trace(go.Scatter(
        x=xy[:, 0], y=xy[:, 1], mode="markers",
        marker=dict(size=3, color="lightgray"),
        customdata=np.arange(len(xy)),
        hovertemplate="node %{customdata}<extra></extra>",
        showlegend=False,
    ))
    fig.update_layout(
        paper_bgcolor="white", plot_bgcolor="white",
        margin=dict(l=0, r=0, t=30, b=0),
        xaxis=dict(visible=False),
        yaxis=dict(visible=False, scaleanchor="x", scaleratio=1),
    )
    return fig

In [ ]:
# Extract 2D camera layout for plotting
# Try COLMAP output first, then fall back to poses.pkl

xy = None

# Option A: Read from COLMAP images.txt in the reconstruction output
colmap_dirs = [
    OUTPUT_ROOT / "results" / "ba_output",
    OUTPUT_ROOT / "results" / "vggt",
    OUTPUT_ROOT / "results" / "merged",
]
for d in colmap_dirs:
    images_txt = d / "images.txt"
    if images_txt.exists():
        try:
            from gtsfm.utils.io import read_images_txt
            poses, _ = read_images_txt(str(images_txt))
            xy = np.array([p.translation()[:2] for p in poses])
            print(f"Loaded {len(poses)} poses from {images_txt}")
            break
        except Exception as e:
            print(f"Failed to read {images_txt}: {e}")

# Option B: Saved poses pickle
if xy is None:
    poses_pkl = OUTPUT_ROOT / "results" / "poses.pkl"
    if poses_pkl.exists():
        from gtsfm.utils.io import load_poses
        poses = load_poses(poses_pkl)
        xy = np.array([p.translation()[:2] for p in poses])
        print(f"Loaded {len(poses)} poses from {poses_pkl}")

if xy is None:
    print("WARNING: No camera poses found. Plotly graph visualizations will be skipped.")
    print("Expected COLMAP output in:", [str(d) for d in colmap_dirs])
else:
    N = len(xy)
    # Extract all edges from cluster tree
    all_edges = list(cluster_tree.all_edges())
    edges_arr = np.array(all_edges, dtype=int)
    valid_mask = (edges_arr[:, 0] < N) & (edges_arr[:, 1] < N)
    edges_arr = edges_arr[valid_mask]
    print(f"Total edges in cluster tree: {len(edges_arr)}, poses: {N}")

In [ ]:
# Visibility graph with bad edges highlighted in red
if xy is not None:
    fig = create_base_figure_with_background(xy, edges_arr)

    bad_edge_arr = np.array(
        [e for e in bad_edges if e[0] < N and e[1] < N], dtype=int
    )
    if len(bad_edge_arr) > 0:
        xe_bad, ye_bad = get_edge_coordinates(xy, bad_edge_arr)
        fig.add_trace(go.Scatter(
            x=xe_bad, y=ye_bad, mode="lines",
            line=dict(width=2, color="red"),
            name="Bad edges", hoverinfo="none",
        ))
        bad_nodes = np.unique(bad_edge_arr.flatten())
        fig.add_trace(go.Scatter(
            x=xy[bad_nodes, 0], y=xy[bad_nodes, 1], mode="markers",
            marker=dict(size=8, color="red", symbol="x"),
            name="Bad edge nodes",
            customdata=bad_nodes,
            hovertemplate="node %{customdata}<extra></extra>",
        ))

    fig.update_layout(title="Visibility Graph with Bad Edges (red)")
    fig.show()
else:
    print("Skipped (no poses available)")

In [ ]:
# Original METIS leaf cluster visualization
if xy is not None:
    leaves = list(cluster_tree.leaves())
    fig = create_base_figure_with_background(xy, edges_arr)

    for idx, leaf in enumerate(leaves):
        name = f"Leaf {idx + 1}"
        nodes = np.array([k for k in leaf.all_keys() if 0 <= k < N], dtype=int)
        if nodes.size == 0:
            continue
        mask = np.isin(edges_arr[:, 0], nodes) & np.isin(edges_arr[:, 1], nodes)
        leaf_edges = edges_arr[mask]
        xe, ye = get_edge_coordinates(xy, leaf_edges)
        fig.add_trace(go.Scatter(
            x=xe, y=ye, mode="lines", line=dict(width=1),
            hoverinfo="none", name=name, legendgroup=name,
        ))
        fig.add_trace(go.Scatter(
            x=xy[nodes, 0], y=xy[nodes, 1], mode="markers",
            marker=dict(size=6), name=name, legendgroup=name,
            showlegend=False, customdata=nodes,
            hovertemplate="node %{customdata}<extra></extra>",
        ))

    fig.update_layout(
        title="Original METIS Leaf Clusters",
        legend=dict(groupclick="togglegroup"),
    )
    fig.show()
else:
    print("Skipped (no poses available)")

## 4. Bad Edges Per Cluster

In [ ]:
leaves = list(cluster_tree.leaves())

labels = []
total_counts = []
bad_counts = []

for idx, leaf in enumerate(leaves):
    leaf_edges = set(leaf.value)
    n_bad = len(leaf_edges & bad_edges)
    labels.append(f"Leaf {idx + 1}")
    total_counts.append(len(leaf_edges))
    bad_counts.append(n_bad)

fig, ax = plt.subplots(figsize=(max(8, len(labels) * 0.8), 5))
x = np.arange(len(labels))
width = 0.35
ax.bar(x - width / 2, total_counts, width, label="Total edges", color="steelblue", alpha=0.7)
ax.bar(x + width / 2, bad_counts, width, label="Bad edges", color="red", alpha=0.7)
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=45, ha="right")
ax.set_ylabel("Edge count")
ax.set_title("Bad Edges per Leaf Cluster")
ax.legend()
plt.tight_layout()
plt.show()

# Print details
for lbl, tot, bad in zip(labels, total_counts, bad_counts):
    pct = 100.0 * bad / tot if tot > 0 else 0
    print(f"  {lbl}: {bad}/{tot} bad ({pct:.0f}%)")

## 5. Re-partition After Pruning Bad Edges
Remove bad edges from the visibility graph and re-run METIS to see how the cluster tree changes.

In [ ]:
from gtsfm.graph_partitioner.metis_partitioner import MetisPartitioner
from gtsfm.products.visibility_graph import prune_edges

original_graph = sorted(cluster_tree.all_edges())
pruned_graph = prune_edges(original_graph, bad_edges)

print(f"Original: {len(original_graph)} edges")
print(f"Pruned:   {len(pruned_graph)} edges (removed {len(original_graph) - len(pruned_graph)})")
print()

partitioner = MetisPartitioner()
pruned_cluster_tree = partitioner.run(pruned_graph)

orig_leaves = list(cluster_tree.leaves())
pruned_leaves = list(pruned_cluster_tree.leaves()) if pruned_cluster_tree else []

print(f"Original: {len(orig_leaves)} leaf clusters")
print(f"Pruned:   {len(pruned_leaves)} leaf clusters")
print()
print("=== ORIGINAL ===")
print(cluster_tree)
print()
print("=== PRUNED ===")
print(pruned_cluster_tree if pruned_cluster_tree else "(empty -- graph may be disconnected)")

## 5b. Connectivity Analysis After Pruning
Analyze connected components resulting from naive edge pruning, visualize them spatially, and compare naive vs. connectivity-preserving strategies.

In [ ]:
import networkx as nx

# Connected component analysis on the naively pruned graph
G_pruned = nx.Graph()
G_pruned.add_edges_from(pruned_graph)

components = sorted(nx.connected_components(G_pruned), key=len, reverse=True)
component_sizes = [len(c) for c in components]

# Map each node to its component index
node_to_component = {}
for comp_idx, comp_nodes in enumerate(components):
    for node in comp_nodes:
        node_to_component[node] = comp_idx

print(f"Connected components after naive pruning: {len(components)}")
print(f"Component sizes: {component_sizes}")
print()

for comp_idx, comp_nodes in enumerate(components):
    sorted_nodes = sorted(comp_nodes)
    if image_filenames:
        labels = [f"{n} ({image_filenames[n]})" if n < len(image_filenames) else str(n) for n in sorted_nodes[:10]]
    else:
        labels = [str(n) for n in sorted_nodes[:10]]
    suffix = f" ... +{len(sorted_nodes) - 10} more" if len(sorted_nodes) > 10 else ""
    print(f"Component {comp_idx} ({len(sorted_nodes)} images): {', '.join(labels)}{suffix}")

# Identify isolated images (in small components)
SMALL_THRESHOLD = 5
small_comps = [c for c in components if len(c) < SMALL_THRESHOLD]
isolated_nodes = set()
for c in small_comps:
    isolated_nodes.update(c)
print(f"\nIsolated images (component < {SMALL_THRESHOLD} nodes): {len(isolated_nodes)}")
for n in sorted(isolated_nodes):
    fname = image_filenames[n] if n < len(image_filenames) else f"idx_{n}"
    comp = node_to_component[n]
    print(f"  Image {n} ({fname}) -- component {comp} (size {component_sizes[comp]})")

In [ ]:
# Plotly: color each connected component on the camera layout
if xy is not None and len(components) > 0:
    import plotly.express as px

    pruned_edges_arr = np.array(pruned_graph, dtype=int)
    valid_mask = (pruned_edges_arr[:, 0] < N) & (pruned_edges_arr[:, 1] < N)
    pruned_edges_arr = pruned_edges_arr[valid_mask]

    fig = create_base_figure_with_background(xy, pruned_edges_arr)

    colors = px.colors.qualitative.Set1 if len(components) <= 9 else px.colors.qualitative.Alphabet
    if len(components) > len(colors):
        colors = colors * ((len(components) // len(colors)) + 1)

    for comp_idx, comp_nodes in enumerate(components):
        comp_arr = np.array([n for n in comp_nodes if 0 <= n < N], dtype=int)
        if comp_arr.size == 0:
            continue

        color = colors[comp_idx % len(colors)]
        name = f"Component {comp_idx} ({len(comp_nodes)} imgs)"
        is_small = len(comp_nodes) < SMALL_THRESHOLD

        # Edges within this component
        mask = np.isin(pruned_edges_arr[:, 0], comp_arr) & np.isin(pruned_edges_arr[:, 1], comp_arr)
        comp_edges = pruned_edges_arr[mask]
        if comp_edges.size > 0:
            xe, ye = get_edge_coordinates(xy, comp_edges)
            fig.add_trace(go.Scatter(
                x=xe, y=ye, mode="lines",
                line=dict(width=1.5, color=color),
                hoverinfo="none", name=name, legendgroup=name,
            ))

        fig.add_trace(go.Scatter(
            x=xy[comp_arr, 0], y=xy[comp_arr, 1], mode="markers",
            marker=dict(size=10 if is_small else 6, color=color,
                        symbol="x" if is_small else "circle"),
            name=name, legendgroup=name, showlegend=(comp_edges.size == 0),
            customdata=comp_arr,
            hovertemplate="node %{customdata}<extra></extra>",
        ))

    fig.update_layout(
        title=f"Connected Components After Naive Pruning ({len(components)} components)",
        legend=dict(groupclick="togglegroup"),
    )
    fig.show()
else:
    print("Skipped (no poses available)")

In [ ]:
# Threshold sweep: how does pruning aggressiveness affect connectivity?
thresholds = np.arange(0.5, 10.5, 0.5)
n_components_list = []
largest_component_list = []
n_removed_list = []
n_images_lost_list = []

all_graph_edges = sorted(cluster_tree.all_edges())
all_nodes = set()
for i, j in all_graph_edges:
    all_nodes.add(i)
    all_nodes.add(j)
total_images = len(all_nodes)

for threshold in thresholds:
    sweep_bad = set()
    for (i, j), stats in edge_quality.items():
        err = stats["mean_reproj_error_px"]
        n_tracks = stats["num_tracks"]
        if n_tracks == 0 or (err != "inf" and err > threshold) or err == "inf":
            sweep_bad.add((i, j))

    swept_graph = [e for e in all_graph_edges if e not in sweep_bad]
    n_removed_list.append(len(all_graph_edges) - len(swept_graph))

    G = nx.Graph()
    G.add_edges_from(swept_graph)
    comps = list(nx.connected_components(G))
    n_components_list.append(len(comps))
    largest = max(len(c) for c in comps) if comps else 0
    largest_component_list.append(largest)
    n_images_lost_list.append(total_images - largest)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].plot(thresholds, n_components_list, "o-", color="steelblue")
axes[0, 0].set_xlabel("Max Reproj Error Threshold (px)")
axes[0, 0].set_ylabel("Number of Components")
axes[0, 0].set_title("Components vs Threshold")
axes[0, 0].axhline(1, color="green", linestyle="--", alpha=0.5, label="Ideal (1)")
axes[0, 0].legend()

axes[0, 1].plot(thresholds, largest_component_list, "o-", color="orange")
axes[0, 1].axhline(total_images, color="green", linestyle="--", alpha=0.5, label=f"All ({total_images})")
axes[0, 1].set_xlabel("Max Reproj Error Threshold (px)")
axes[0, 1].set_ylabel("Largest Component (images)")
axes[0, 1].set_title("Largest Component vs Threshold")
axes[0, 1].legend()

axes[1, 0].plot(thresholds, n_removed_list, "o-", color="red")
axes[1, 0].set_xlabel("Max Reproj Error Threshold (px)")
axes[1, 0].set_ylabel("Edges Removed")
axes[1, 0].set_title("Edges Removed vs Threshold")

axes[1, 1].plot(thresholds, n_images_lost_list, "o-", color="purple")
axes[1, 1].set_xlabel("Max Reproj Error Threshold (px)")
axes[1, 1].set_ylabel("Images Lost")
axes[1, 1].set_title("Images Lost vs Threshold")
axes[1, 1].axhline(0, color="green", linestyle="--", alpha=0.5)

plt.suptitle("Naive Pruning: Connectivity Impact by Threshold", fontsize=14)
plt.tight_layout()
plt.show()

print(f"{'Threshold':>10} {'Removed':>8} {'Components':>11} {'Largest':>8} {'Lost':>6}")
print("-" * 48)
for t, r, c, l, lost in zip(thresholds, n_removed_list, n_components_list, largest_component_list, n_images_lost_list):
    print(f"{t:>10.1f} {r:>8d} {c:>11d} {l:>8d} {lost:>6d}")

In [ ]:
# Connectivity-preserving vs naive pruning comparison
from gtsfm.products.visibility_graph import prune_edges_preserve_connectivity

preserved_graph, removed_edges, kept_bridges = prune_edges_preserve_connectivity(
    all_graph_edges, bad_edges
)

G_naive = nx.Graph()
G_naive.add_edges_from(pruned_graph)
naive_comps = sorted(nx.connected_components(G_naive), key=len, reverse=True)

G_preserved = nx.Graph()
G_preserved.add_edges_from(preserved_graph)
preserved_comps = sorted(nx.connected_components(G_preserved), key=len, reverse=True)

print("=== Naive Pruning ===")
print(f"  Edges removed:  {len(all_graph_edges) - len(pruned_graph)}")
print(f"  Components:     {len(naive_comps)}")
print(f"  Component sizes: {[len(c) for c in naive_comps]}")
print()
print("=== Connectivity-Preserving Pruning ===")
print(f"  Edges removed:  {len(removed_edges)}")
print(f"  Bridges kept:   {len(kept_bridges)}")
print(f"  Components:     {len(preserved_comps)}")
print(f"  Component sizes: {[len(c) for c in preserved_comps]}")
print()

if kept_bridges:
    print("Bridge edges kept (bad but necessary for connectivity):")
    for i, j in sorted(kept_bridges):
        stats = edge_quality.get((i, j), {})
        err = stats.get("mean_reproj_error_px", "?")
        tracks = stats.get("num_tracks", "?")
        err_str = f"{err:.1f}" if isinstance(err, (int, float)) and err != float("inf") else str(err)
        print(f"  ({i},{j}): err={err_str}px, tracks={tracks}")
else:
    print("No bridge edges needed -- all bad edges were safely removable.")

In [ ]:
# Pruned cluster visualization
if xy is not None and pruned_cluster_tree is not None:
    pruned_edges_arr = np.array(pruned_graph, dtype=int)
    valid_mask = (pruned_edges_arr[:, 0] < N) & (pruned_edges_arr[:, 1] < N)
    pruned_edges_arr = pruned_edges_arr[valid_mask]

    fig = create_base_figure_with_background(xy, pruned_edges_arr)

    for idx, leaf in enumerate(pruned_cluster_tree.leaves()):
        name = f"Leaf {idx + 1}"
        nodes = np.array([k for k in leaf.all_keys() if 0 <= k < N], dtype=int)
        if nodes.size == 0:
            continue
        mask = np.isin(pruned_edges_arr[:, 0], nodes) & np.isin(pruned_edges_arr[:, 1], nodes)
        leaf_edges = pruned_edges_arr[mask]
        xe, ye = get_edge_coordinates(xy, leaf_edges)
        fig.add_trace(go.Scatter(
            x=xe, y=ye, mode="lines", line=dict(width=1),
            hoverinfo="none", name=name, legendgroup=name,
        ))
        fig.add_trace(go.Scatter(
            x=xy[nodes, 0], y=xy[nodes, 1], mode="markers",
            marker=dict(size=6), name=name, legendgroup=name,
            showlegend=False, customdata=nodes,
            hovertemplate="node %{customdata}<extra></extra>",
        ))

    fig.update_layout(
        title="Pruned Visibility Graph - Leaf Clusters",
        legend=dict(groupclick="togglegroup"),
    )
    fig.show()
else:
    print("Skipped (no poses or empty pruned tree)")

## 6. Original vs Pruned Pipeline Comparison
Compare edge quality from the original run against the pruned run to measure the impact of removing bad edges.

In [ ]:
# Load pruned run results
pruned_report = None
pruned_edge_quality = {}
pruned_bad_edges = set()
pruned_metadata = {}

if PRUNED_EDGE_QUALITY_JSON.exists():
    with open(PRUNED_EDGE_QUALITY_JSON) as f:
        pruned_report = json.load(f)

    for edge_str, stats in pruned_report["edge_quality"].items():
        i, j = map(int, edge_str.strip("()").split(","))
        pruned_edge_quality[(i, j)] = stats

    for s in pruned_report["bad_edges"]:
        i, j = map(int, s.strip("()").split(","))
        pruned_bad_edges.add((i, j))

    pruned_metadata = pruned_report["metadata"]
    print(f"Pruned run — Total edges: {pruned_metadata['total_edges']}")
    print(f"Pruned run — Bad edges:   {pruned_metadata['bad_edge_count']}")
    print(f"Pruned run — Zero-track:  {pruned_metadata['edges_with_no_tracks']}")
else:
    print(f"Pruned report not found at {PRUNED_EDGE_QUALITY_JSON}")
    print("Run the pipeline with --edge_quality_json to generate it.")

In [ ]:
# Side-by-side summary comparison
if pruned_report is not None:
    # Compute stats for original
    orig_errors = [s["mean_reproj_error_px"] for s in edge_quality.values() if s["mean_reproj_error_px"] != "inf"]
    pruned_errors = [s["mean_reproj_error_px"] for s in pruned_edge_quality.values() if s["mean_reproj_error_px"] != "inf"]

    rows = [
        ("Total edges", metadata["total_edges"], pruned_metadata["total_edges"]),
        ("Bad edges", metadata["bad_edge_count"], pruned_metadata["bad_edge_count"]),
        ("Zero-track edges", metadata["edges_with_no_tracks"], pruned_metadata["edges_with_no_tracks"]),
        ("Mean reproj error (px)", f"{np.mean(orig_errors):.2f}", f"{np.mean(pruned_errors):.2f}" if pruned_errors else "N/A"),
        ("Median reproj error (px)", f"{np.median(orig_errors):.2f}", f"{np.median(pruned_errors):.2f}" if pruned_errors else "N/A"),
        ("Max reproj error (px)", f"{np.max(orig_errors):.2f}", f"{np.max(pruned_errors):.2f}" if pruned_errors else "N/A"),
    ]

    print(f"{'Metric':<28} {'Original':>12} {'Pruned':>12} {'Delta':>12}")
    print("-" * 66)
    for label, orig, pruned in rows:
        try:
            delta = float(pruned) - float(orig)
            delta_str = f"{delta:+.2f}" if isinstance(orig, str) else f"{delta:+d}"
        except (ValueError, TypeError):
            delta_str = ""
        print(f"{label:<28} {str(orig):>12} {str(pruned):>12} {delta_str:>12}")
else:
    print("Skipped (no pruned report)")

In [ ]:
# Overlaid error distribution histograms
if pruned_report is not None:
    orig_errors = np.array([s["mean_reproj_error_px"] for s in edge_quality.values() if s["mean_reproj_error_px"] != "inf"])
    pruned_errors = np.array([s["mean_reproj_error_px"] for s in pruned_edge_quality.values() if s["mean_reproj_error_px"] != "inf"])

    fig, ax = plt.subplots(figsize=(10, 5))
    bins = np.linspace(0, max(orig_errors.max(), pruned_errors.max() if len(pruned_errors) > 0 else 0) * 1.05, 50)
    ax.hist(orig_errors, bins=bins, alpha=0.5, label=f"Original ({len(orig_errors)} edges)", color="steelblue", edgecolor="black")
    ax.hist(pruned_errors, bins=bins, alpha=0.5, label=f"Pruned ({len(pruned_errors)} edges)", color="orange", edgecolor="black")
    ax.axvline(5.0, color="red", linestyle="--", linewidth=1.5, label="Bad threshold (5px)")
    ax.set_xlabel("Mean Reprojection Error (px)")
    ax.set_ylabel("Count")
    ax.set_title("Error Distribution: Original vs Pruned")
    ax.legend()
    plt.tight_layout()
    plt.show()
else:
    print("Skipped (no pruned report)")

In [ ]:
# Per-edge error change scatter plot (edges present in both runs)
if pruned_report is not None:
    common_edges = set(edge_quality.keys()) & set(pruned_edge_quality.keys())
    print(f"Edges in both runs: {len(common_edges)}")

    orig_vals, pruned_vals, was_bad = [], [], []
    for e in common_edges:
        o = edge_quality[e]["mean_reproj_error_px"]
        p = pruned_edge_quality[e]["mean_reproj_error_px"]
        if o == "inf" or p == "inf":
            continue
        orig_vals.append(o)
        pruned_vals.append(p)
        was_bad.append(e in bad_edges)

    orig_vals = np.array(orig_vals)
    pruned_vals = np.array(pruned_vals)
    was_bad = np.array(was_bad)

    fig, ax = plt.subplots(figsize=(8, 8))
    ax.scatter(orig_vals[~was_bad], pruned_vals[~was_bad], alpha=0.4, s=15, c="steelblue", label="Was good")
    ax.scatter(orig_vals[was_bad], pruned_vals[was_bad], alpha=0.6, s=25, c="red", marker="x", label="Was bad")
    lim = max(orig_vals.max(), pruned_vals.max()) * 1.05
    ax.plot([0, lim], [0, lim], "k--", linewidth=0.8, alpha=0.5, label="No change (y=x)")
    ax.set_xlabel("Original reproj error (px)")
    ax.set_ylabel("Pruned reproj error (px)")
    ax.set_title("Per-Edge Error: Original vs Pruned")
    ax.legend()
    ax.set_aspect("equal")
    plt.tight_layout()
    plt.show()

    improved = np.sum(pruned_vals < orig_vals - 0.5)
    worsened = np.sum(pruned_vals > orig_vals + 0.5)
    similar = len(orig_vals) - improved - worsened
    print(f"Improved (> 0.5px better): {improved}")
    print(f"Worsened (> 0.5px worse):  {worsened}")
    print(f"Similar (within 0.5px):    {similar}")
else:
    print("Skipped (no pruned report)")

In [ ]:
# Bad edge set comparison: fixed, persistent, new
if pruned_report is not None:
    # Edges in both runs
    common_edges = set(edge_quality.keys()) & set(pruned_edge_quality.keys())

    fixed = bad_edges - pruned_bad_edges  # were bad, now good (or removed entirely)
    persistent = bad_edges & pruned_bad_edges  # bad in both
    new_bad = pruned_bad_edges - bad_edges  # were good, now bad

    print(f"Fixed (bad -> good):       {len(fixed)}")
    print(f"Persistent (bad -> bad):   {len(persistent)}")
    print(f"New bad (good -> bad):     {len(new_bad)}")
    print()

    if new_bad:
        print("NEW bad edges (regressions from pruning):")
        for e in sorted(new_bad):
            stats = pruned_edge_quality.get(e, {})
            err = stats.get("mean_reproj_error_px", "?")
            tracks = stats.get("num_tracks", "?")
            err_str = f"{err:.1f}" if isinstance(err, (int, float)) else err
            print(f"  ({e[0]},{e[1]}): err={err_str}px, tracks={tracks}")
    else:
        print("No new bad edges introduced by pruning.")
else:
    print("Skipped (no pruned report)")

In [ ]:
# Per-cluster bad edge comparison for the pruned run
if pruned_report is not None and PRUNED_CLUSTER_TREE_PKL.exists():
    with open(PRUNED_CLUSTER_TREE_PKL, "rb") as f:
        pruned_cluster_tree_loaded = pickle.load(f)

    pruned_leaves = list(pruned_cluster_tree_loaded.leaves())

    labels = []
    total_counts = []
    bad_counts = []
    for idx, leaf in enumerate(pruned_leaves):
        leaf_edges = set(leaf.value)
        n_bad = len(leaf_edges & pruned_bad_edges)
        labels.append(f"Leaf {idx + 1}")
        total_counts.append(len(leaf_edges))
        bad_counts.append(n_bad)

    fig, ax = plt.subplots(figsize=(max(8, len(labels) * 0.8), 5))
    x = np.arange(len(labels))
    width = 0.35
    ax.bar(x - width / 2, total_counts, width, label="Total edges", color="steelblue", alpha=0.7)
    ax.bar(x + width / 2, bad_counts, width, label="Bad edges", color="red", alpha=0.7)
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=45, ha="right")
    ax.set_ylabel("Edge count")
    ax.set_title("Pruned Run: Bad Edges per Leaf Cluster")
    ax.legend()
    plt.tight_layout()
    plt.show()

    for lbl, tot, bad in zip(labels, total_counts, bad_counts):
        pct = 100.0 * bad / tot if tot > 0 else 0
        print(f"  {lbl}: {bad}/{tot} bad ({pct:.0f}%)")
elif pruned_report is not None:
    print(f"Pruned cluster tree not found at {PRUNED_CLUSTER_TREE_PKL}")
else:
    print("Skipped (no pruned report)")